# DB2-Salesforce populate survey results obtain from DB2

In [1]:
# Parameters
# API settings
api_url = '/services/data/v43.0/sobjects'
external_id = 'nanoHUB_user_ID__c'
object_id = 'Contact'

# login parameters to be handled by Papermill
from nanoHUB.application import Application

application = Application.get_instance()
nanohub_db = application.new_db_engine('nanohub')
nanohub_metrics_db = application.new_db_engine('nanohub_metrics')
wang159_myrmekes_db = application.new_db_engine('wang159_myrmekes')


salesforce = application.new_salesforce_engine()
db_s = salesforce

nanoHUB - Serving Students, Researchers & Instructors


Obtained Salesforce access token ...... True


In [2]:
import pandas as pd
import datetime

## Get the URL code from DB2


In [3]:
# page_id indicates the question
survey_df = pd.read_sql_query("select user_id,input from jos_reply_replies;",nanohub_db) #limit 5000

In [4]:
display(survey_df.head(2))

,user_id,input
0,170942,"{""text"":""Test.""}"
1,206944,"{""text"":""NCN staff test.""}"


In [5]:
## some processing
s_temp = [i.split('":"')[-1][:-2] for i in survey_df['input'].to_list()]

survey_df['NH_survey_results__c'] = s_temp

In [6]:
display(survey_df.head(2))
display(survey_df.tail(2))

,user_id,input,NH_survey_results__c
0,170942,"{""text"":""Test.""}",Test.
1,206944,"{""text"":""NCN staff test.""}",NCN staff test.


,user_id,input,NH_survey_results__c
85,183731,"{""text"":""Dear NanoHub Developers, \r\n\r\n\r\n...","Dear NanoHub Developers, \r\n\r\n\r\nI teach C..."
86,36138,"{""text"":""using for self interest\r\n\r\nif I ...",using for self interest\r\n\r\nif I apply for...


In [7]:
survey_df = survey_df.drop(columns=['input'])

In [8]:
display(survey_df.head(5))

,user_id,NH_survey_results__c
0,170942,Test.
1,206944,NCN staff test.
2,3482,"This is a test - Gerhard Klimeck, \r\nHenry Bu..."
3,3482,"Well, let's see what happens when I type anoth..."
4,279076,We (the group of Prof. James Glazier at Indian...


In [9]:
survey_df = survey_df.rename(columns={'user_id':'nanoHUB_user_ID__c'})
display(survey_df.head(2))

,nanoHUB_user_ID__c,NH_survey_results__c
0,170942,Test.
1,206944,NCN staff test.


In [10]:
## check for duplicates, if they exist, then merge
duplicates = pd.concat(g for _, g in survey_df.groupby("nanoHUB_user_ID__c") if len(g) > 1)

display(duplicates)

,nanoHUB_user_ID__c,NH_survey_results__c
2,3482,"This is a test - Gerhard Klimeck, \r\nHenry Bu..."
3,3482,"Well, let's see what happens when I type anoth..."
49,164228,Wow- what a lot of questions! I don\u2019t k ...
50,164228,Wow- what a lot of questions! I don\u2019t k ...
51,164228,Did you preview this or get any feedback befor...
83,164228,This is Tanya on 5\/6\/2021 submitting somethi...
1,206944,NCN staff test.
6,206944,Test 4\/23


In [11]:
nh_ids = duplicates['nanoHUB_user_ID__c'].to_list()
nh_results = duplicates['NH_survey_results__c'].to_list()

In [12]:
from collections import Counter

In [13]:
# build new array
dict_nh_ids = Counter(nh_ids)

nh_uniques_ids = [i for i in dict_nh_ids.keys()]
nh_uniques_freq = [i for i in dict_nh_ids.values()]

In [14]:
survey_res_uniques = []
dups_list = duplicates['NH_survey_results__c'].to_list()

counter = 0
for i,j in enumerate(nh_uniques_freq):
    temp_survey_res = []
    for k in range(j):
        temp_survey_res.append(dups_list[counter+k])
        
    survey_res_uniques.append('\n '.join(temp_survey_res) )
    
    counter += j

In [15]:
dups_df = pd.DataFrame()
dups_df['nanoHUB_user_ID__c'] = nh_uniques_ids
dups_df['NH_survey_results__c'] = survey_res_uniques
display(dups_df.head(5))

,nanoHUB_user_ID__c,NH_survey_results__c
0,3482,"This is a test - Gerhard Klimeck, \r\nHenry Bu..."
1,164228,Wow- what a lot of questions! I don\u2019t k ...
2,206944,NCN staff test.\n Test 4\/23


In [16]:
## send both survey_df and dups_df; dups after survey 

## Send to SF

In [17]:
db_s.object_id = object_id
db_s.external_id = external_id

# send data to Salesforce
db_s.send_data(survey_df)

[Success] Bulk job creation successful. Job ID = 7505w00000b1owUAAQ
hello


[Success] CSV upload successful. Job ID = 7505w00000b1owUAAQ


[Success] Closing job successful. Job ID = 7505w00000b1owUAAQ


In [18]:
# check status
db_s.check_bulk_status()

{'id': '7505w00000b1owUAAQ',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-22T12:09:38.000+0000',
 'systemModstamp': '2021-10-22T12:09:39.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [19]:
# check status
from pprint import pprint

pprint(db_s.check_bulk_failed_results())

''


In [20]:
## second send for dups_df

In [21]:
# create DB2 to Salesforce API object
db_2 = salesforce

In [22]:
db_2.object_id = object_id
db_2.external_id = external_id

# send data to Salesforce
db_2.send_data(dups_df)

[Success] Bulk job creation successful. Job ID = 7505w00000b1oweAAA
hello


[Success] CSV upload successful. Job ID = 7505w00000b1oweAAA


[Success] Closing job successful. Job ID = 7505w00000b1oweAAA


In [23]:
# check status
db_2.check_bulk_status()

{'id': '7505w00000b1oweAAA',
 'operation': 'upsert',
 'object': 'Contact',
 'createdById': '0055w00000DM5bOAAT',
 'createdDate': '2021-10-22T12:09:41.000+0000',
 'systemModstamp': '2021-10-22T12:09:42.000+0000',
 'state': 'UploadComplete',
 'externalIdFieldName': 'nanoHUB_user_ID__c',
 'concurrencyMode': 'Parallel',
 'contentType': 'CSV',
 'apiVersion': 47.0,
 'jobType': 'V2Ingest',
 'lineEnding': 'LF',
 'columnDelimiter': 'COMMA',
 'numberRecordsProcessed': 0,
 'numberRecordsFailed': 0,
 'retries': 0,
 'totalProcessingTime': 0,
 'apiActiveProcessingTime': 0,
 'apexProcessingTime': 0}

In [24]:
# check status
pprint(db_2.check_bulk_failed_results())

''
